In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from transformers import ViTFeatureExtractor, ViTForImageClassification

In [ ]:
# # Define a dataset class for WSI image classification
# class WSIDataset(Dataset):
#     def __init__(self, data_dir, csv_file, transform=None):
#         self.data_dir = data_dir
#         self.data_info = pd.read_csv(csv_file)
#         self.transform = transform

#     def __len__(self):
#         return len(self.data_info)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.data_dir, self.data_info.iloc[idx, 0])
#         image = Image.open(img_name)
#         label = self.data_info.iloc[idx, 1]

#         if self.transform:
#             image = self.transform(image)

#         return image, label

In [ ]:
# Define the model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# Load the model onto the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
import torchvision

train_data_dir = '/content/drive/MyDrive/WSI_Dataset/Train'  # Replace with your actual path
train_csv_file = '/content/drive/MyDrive/WSI_Dataset/Train/train.csv'  # Replace with your actual path

val_data_dir = '/content/drive/MyDrive/WSI_Dataset/Validate'  # Replace with your actual path
val_csv_file = '/content/drive/MyDrive/WSI_Dataset/Validate/validate.csv'  # Replace with your actual path

test_data_dir = '/content/drive/MyDrive/WSI_Dataset/Test'  # Replace with your actual path
test_csv_file = '/content/drive/MyDrive/WSI_Dataset/Test/test.csv'  # Replace with your actual path

train_dataset = torchvision.datasets.ImageFolder(root=train_data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# val_dataset = WSIDataset(data_dir=val_data_dir, csv_file=val_csv_fval_dataset = WSIDataset(data_dir=val_data_dir, csv_file=val_csv_file, transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=32)

# test_dataset = WSIDataset(data_dir=test_data_dir, csv_file=test_csv_file, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=32)ile, transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=32)

# test_dataset = WSIDataset(data_dir=test_data_dir, csv_file=test_csv_file, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
import zipfile

# Specify the path to the ZIP file
zip_file_path = "/content/drive/MyDrive/WSI_Dataset/Train/TCGA-A6-2671.zip"

# Specify the directory where you want to extract the contents
extracted_dir = "/content/drive/MyDrive/WSI_Dataset/Train/TCGA-A6-2671"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

print("File has been successfully unzipped to:", extracted_dir)


File has been successfully unzipped to: /content/drive/MyDrive/WSI_Dataset/Train/TCGA-A6-2671


In [10]:
# Training loop
num_epochs = 10  # Replace with the desired number of epochs
learning_rate = 0.001  # Adjust as needed
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# Validation loop
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()


print(f'Validation Accuracy: {val_accuracy * 10:.2f}%')

Validation Accuracy: 82.30%
